# Conrad et al., 2023 POD Method

Calculate probability of detection using the POD method outlined in Conrad et al., 2023


In [14]:
# First, code for generating wind data - example code
from methods_source import summarize_wind_conditions, calc_1min_windspeed
import datetime

test_1 = datetime.datetime(2022, 10, 10, 18, 16, 10)
test_2 = datetime.datetime(2022, 10, 10, 18, 17, 10)

wind_test1 = summarize_wind_conditions(test_1, test_2)

wind_test2 = calc_1min_windspeed(test_1, direction='forward')

{'average_windspeed': 1.0174869655737704, 'average_winddirection': 116.53198852459018, 'stdev_windspeed': 0.2118695722622119, 'stdev_winddirection': 30.928243664269548}


1.0174869655737704

In [8]:
from methods_source import load_overpass_summary, feet_per_meter, calc_1min_windspeed
import pandas as pd
kairos = load_overpass_summary('Kairos', 1, strict_discard=False)
feet_per_meter = feet_per_meter()

kairos_POD_variables = pd.DataFrame()
kairos_POD_variables['overpass_id'] = kairos['overpass_id']
kairos_POD_variables['overpass_datetime'] = kairos['overpass_datetime']
kairos_POD_variables['h'] = kairos['altitude_feet'] / feet_per_meter # flight altitude in m
kairos_POD_variables['u'] = kairos['overpass_datetime'].apply(calc_1min_windspeed) # windspeed in m/s
kairos_POD_variables['Q'] = kairos['release_rate_kgh'] # release rate in kg / hr
kairos_POD_variables['D'] = kairos['operator_detected'].apply(int) # operator detected as 0 or 1

def make_predictor_function(h, u, Q):
    def my_predictor(phi1, phi2, phi3, phi4, phi5, phi6):
        return (phi1 * (Q - phi2) ** phi3) / (((h/1000)**phi4) * ((u - phi5) ** phi6))

    return my_predictor
# Next:
# Generate "my_predictor" for each overpass
# Generate Di ln(Fi) + (1 - Di) ln(1 - Fi) for each overpass
# Sum the column with


     overpass_id   overpass_datetime           h         u           Q  D
0            1.0 2022-10-24 16:46:15  408.871167  2.666187    0.000000  0
1            2.0 2022-10-24 16:49:22  416.461333  2.564415    0.000000  0
2            3.0 2022-10-24 16:52:32  398.889667  4.180655   18.748945  1
3            4.0 2022-10-24 16:55:42  403.756000  3.860501   18.719447  1
4            5.0 2022-10-24 16:59:01  409.230833  3.003148   18.739307  1
..           ...                 ...         ...       ...         ... ..
344        345.0 2022-10-28 20:27:06  409.913000  2.439459  392.153592  1
345        346.0 2022-10-28 20:30:12  407.326667  2.131373  382.078576  1
346        347.0 2022-10-28 20:33:34  415.323667  3.115726  381.475330  1
347        348.0 2022-10-28 20:36:41  422.359500  3.774662   25.689220  1
348        349.0 2022-10-28 20:39:57  408.460333  1.909657  113.677707  1

[349 rows x 6 columns]


In [6]:
print(kairos.operator_detected.apply(int))

0      0
1      0
2      1
3      1
4      1
      ..
344    1
345    1
346    1
347    1
348    1
Name: operator_detected, Length: 349, dtype: int64
